In [1]:
import numpy as np
import pandas as pd

In [10]:
#load the csv file
dataset = pd.read_csv('/content/diabetes_dataset.csv' , sep=',')
dataset.head(5)

,6,148,72,35,0,33.6,0.627,50,1
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


In [11]:
dataset.shape

(767, 9)

In [12]:
# Split the data into input X and output Y
X = dataset.iloc[:, 0:8]
Y = dataset.iloc[:,8]

In [13]:
print(X)

      6  148  72  35    0  33.6  0.627  50
0     1   85  66  29    0  26.6  0.351  31
1     8  183  64   0    0  23.3  0.672  32
2     1   89  66  23   94  28.1  0.167  21
3     0  137  40  35  168  43.1  2.288  33
4     5  116  74   0    0  25.6  0.201  30
..   ..  ...  ..  ..  ...   ...    ...  ..
762  10  101  76  48  180  32.9  0.171  63
763   2  122  70  27    0  36.8  0.340  27
764   5  121  72  23  112  26.2  0.245  30
765   1  126  60   0    0  30.1  0.349  47
766   1   93  70  31    0  30.4  0.315  23

[767 rows x 8 columns]


In [14]:
print(Y)

0      0
1      1
2      0
3      1
4      0
      ..
762    0
763    0
764    0
765    1
766    0
Name: 1, Length: 767, dtype: int64


In [15]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [37]:
# Defining Keras Model
model = Sequential([
    Dense(units = 18, input_shape = (8,), activation = 'relu'),
    Dense(units = 9, activation = 'relu'),
    Dense(units = 1, activation = 'sigmoid')
])

In [38]:
# Compile Keras Model
model.compile(loss = 'binary_crossentropy', optimizer = 'adam' , metrics = ['accuracy'])


In [39]:
# Fit Keras Model
model.fit(X, Y, epochs =150, batch_size = 10)

Epoch 1/150
77/77 [==============================] - 1s 2ms/step - loss: 4.4806 - accuracy: 0.6402
Epoch 2/150
77/77 [==============================] - 0s 2ms/step - loss: 0.9143 - accuracy: 0.6415
Epoch 3/150
77/77 [==============================] - 0s 2ms/step - loss: 0.8093 - accuracy: 0.6519
Epoch 4/150
77/77 [==============================] - 0s 2ms/step - loss: 0.7943 - accuracy: 0.6519
Epoch 5/150
77/77 [==============================] - 0s 2ms/step - loss: 0.7669 - accuracy: 0.6597
Epoch 6/150
77/77 [==============================] - 0s 2ms/step - loss: 0.7107 - accuracy: 0.6688
Epoch 7/150
77/77 [==============================] - 0s 2ms/step - loss: 0.7207 - accuracy: 0.6649
Epoch 8/150
77/77 [==============================] - 0s 2ms/step - loss: 0.6899 - accuracy: 0.6675
Epoch 9/150
77/77 [==============================] - 0s 2ms/step - loss: 0.6743 - accuracy: 0.6610
Epoch 10/150
77/77 [==============================] - 0s 2ms/step - loss: 0.6825 - accuracy: 0.6714
Epoch 11/

In [41]:
#Evaluate the keras model
_, accuracy = model.evaluate(X,Y)
print('Accuracy : %.2f'%(accuracy*100))

24/24 [==============================] - 0s 2ms/step - loss: 0.4696 - accuracy: 0.7901
Accuracy : 79.01


In [20]:
#Make predicitions
predictions = model.predict(X)
#round predictions
rounded =  [round(x[0]) for x in predictions]

24/24 [==============================] - 0s 1ms/step


In [21]:
#class prediction
prediction = (model.predict(X)>0.5).astype(int)

24/24 [==============================] - 0s 2ms/step


In [23]:
!pip install --quiet optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.1 MB/s eta 0:00:00


In [30]:
import optuna
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

In [32]:
def objective(trial):
  #define hyperparameters to search
  units = trial.suggest_loguniform('units', 8,20)
  units2= trial.suggest_loguniform('units',8,20)
  # train the model
  model = Sequential([
    Dense(units = units, input_shape = (8,), activation = 'relu'),
    Dense(units = units2, activation = 'relu'),
    Dense(units = 1, activation = 'sigmoid')
    ])
  model.compile(loss = 'binary_crossentropy', optimizer = 'adam' , metrics = ['accuracy'])
  model.fit(X,Y, epochs = 150, batch_size = 10)

  #make predictions on the test set and calculate accuracy
  y_pred = model.predict(X)
  auc_roc = roc_auc_score(Y, y_pred)

  return accuracy

#Run Hyperparameter optimization with Optuna
study = optuna.create_study(direction = 'minimize')
study.optimize(objective, n_trials =150)

#Train the model with the best hyperparamters found by Optuna
best_params = study.best_params
model = Sequential([
    Dense(units = best_params['units'], input_shape = (8,), activation = 'relu'),
    Dense(units = best_params['units2'], activation = 'relu'),
    Dense(units = 1, activation = 'sigmoid')
    ])
model.fit(X,Y)

#Make predictions on the test set and calculate accuracy
y_pred = (model.predict(X)>0.5).astype(int)
accuracy = accuracy_score(Y, y_pred)

#print the best parameters
print('Best Parameters: ', best_params)
print('Accuracy : ',auc_roc)


[I 2023-06-27 11:10:44,458] A new study created in memory with name: no-name-5d338de3-4914-487b-b884-e72809c4378f


Epoch 1/150


<ipython-input-32-3a496f30997f>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  units = trial.suggest_loguniform('units', 8,20)
<ipython-input-32-3a496f30997f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  units2= trial.suggest_loguniform('units',8,20)


77/77 [==============================] - 1s 2ms/step - loss: 22.8961 - accuracy: 0.6519
Epoch 2/150
77/77 [==============================] - 0s 2ms/step - loss: 9.5386 - accuracy: 0.6519
Epoch 3/150
77/77 [==============================] - 0s 2ms/step - loss: 2.7493 - accuracy: 0.5619
Epoch 4/150
77/77 [==============================] - 0s 2ms/step - loss: 1.3084 - accuracy: 0.5189
Epoch 5/150
77/77 [==============================] - 0s 2ms/step - loss: 0.9476 - accuracy: 0.5593
Epoch 6/150
77/77 [==============================] - 0s 2ms/step - loss: 0.8140 - accuracy: 0.6063
Epoch 7/150
77/77 [==============================] - 0s 2ms/step - loss: 0.7377 - accuracy: 0.6245
Epoch 8/150
77/77 [==============================] - 0s 2ms/step - loss: 0.7008 - accuracy: 0.6336
Epoch 9/150
77/77 [==============================] - 0s 2ms/step - loss: 0.6735 - accuracy: 0.6454
Epoch 10/150
77/77 [==============================] - 0s 2ms/step - loss: 0.6522 - accuracy: 0.6597
Epoch 11/150
77/77 [

[I 2023-06-27 11:11:09,585] Trial 0 finished with value: 0.7731420993804932 and parameters: {'units': 9.837957910838002}. Best is trial 0 with value: 0.7731420993804932.


...

Epoch 1/150


<ipython-input-32-3a496f30997f>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  units = trial.suggest_loguniform('units', 8,20)
<ipython-input-32-3a496f30997f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  units2= trial.suggest_loguniform('units',8,20)


77/77 [==============================] - 1s 2ms/step - loss: 5.6873 - accuracy: 0.6180
Epoch 2/150
77/77 [==============================] - 0s 2ms/step - loss: 1.2177 - accuracy: 0.6362
Epoch 3/150
77/77 [==============================] - 0s 2ms/step - loss: 0.8558 - accuracy: 0.6610
Epoch 4/150
77/77 [==============================] - 0s 2ms/step - loss: 0.7919 - accuracy: 0.6362
Epoch 5/150
77/77 [==============================] - 0s 2ms/step - loss: 0.6786 - accuracy: 0.6688
Epoch 6/150
77/77 [==============================] - 0s 2ms/step - loss: 0.6764 - accuracy: 0.6558
Epoch 7/150
77/77 [==============================] - 0s 2ms/step - loss: 0.6199 - accuracy: 0.6636
Epoch 8/150
77/77 [==============================] - 0s 2ms/step - loss: 0.6625 - accuracy: 0.6688
Epoch 9/150
77/77 [==============================] - 0s 2ms/step - loss: 0.6169 - accuracy: 0.6714
Epoch 10/150
77/77 [==============================] - 0s 2ms/step - loss: 0.6353 - accuracy: 0.6767
Epoch 11/150
77/77 [=

[W 2023-06-27 11:12:02,390] Trial 2 failed with parameters: {'units': 13.152904250779445} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-32-3a496f30997f>", line 12, in objective
    model.fit(X,Y, epochs = 150, batch_size = 10)
  File "/usr/local/lib/python3.10/dist-packages/keras/utils/traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/engine/training.py", line 1685, in fit
    tmp_logs = self.train_function(iterator)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/polymorphic_function/polymorphic_function.py", line 894, in __call__
    res

KeyboardInterrupt: ignored